# OneLake Introduction and Capabilities

This notebook demonstrates the key capabilities of Microsoft Fabric OneLake - the unified data lake for the entire organization.

## What is OneLake?

OneLake is a unified, logical data lake for your entire organization. It provides:
- **Single storage location** for all your analytics data
- **Automatic integration** with all Fabric workloads
- **Delta Parquet format** for efficient storage and querying
- **Lakehouse architecture** combining data lake and data warehouse capabilities
- **Shortcuts** to external data sources without data movement

## Prerequisites

To run this notebook, you need:
- A Microsoft Fabric workspace
- A Lakehouse created in your workspace
- Appropriate permissions to read/write data

## 1. Connecting to OneLake

In Fabric notebooks, you're automatically connected to OneLake. Let's verify the connection and explore the environment.

In [ ]:
# Import necessary libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Display Spark configuration
print(f"Spark Version: {spark.version}")
print(f"Application Name: {spark.sparkContext.appName}")

## 2. Reading Data from OneLake

OneLake supports multiple file formats. Let's demonstrate reading different formats:

In [ ]:
# Example: Reading CSV data from OneLake
# Replace with your actual path
# csv_path = "Files/sample_data.csv"
# df_csv = spark.read.csv(csv_path, header=True, inferSchema=True)
# display(df_csv.limit(10))

print("To read CSV files, use: spark.read.csv(path, header=True, inferSchema=True)")

In [ ]:
# Example: Reading Parquet data from OneLake
# parquet_path = "Files/sample_data.parquet"
# df_parquet = spark.read.parquet(parquet_path)
# display(df_parquet.limit(10))

print("To read Parquet files, use: spark.read.parquet(path)")

In [ ]:
# Example: Reading Delta tables from OneLake
# delta_table_path = "Tables/sales_data"
# df_delta = spark.read.format("delta").load(delta_table_path)
# display(df_delta.limit(10))

print("To read Delta tables, use: spark.read.format('delta').load(path)")

## 3. Creating Sample Data

Let's create some sample data to demonstrate OneLake capabilities:

In [ ]:
# Create sample sales data
from datetime import datetime, timedelta
import random

# Generate sample data
sample_data = []
start_date = datetime(2024, 1, 1)
products = ['Laptop', 'Phone', 'Tablet', 'Monitor', 'Keyboard']
regions = ['North', 'South', 'East', 'West']

for i in range(100):
    sample_data.append((
        i + 1,
        random.choice(products),
        random.choice(regions),
        random.randint(1, 10),
        round(random.uniform(100, 2000), 2),
        start_date + timedelta(days=random.randint(0, 365))
    ))

# Define schema
schema = StructType([
    StructField("OrderID", IntegerType(), False),
    StructField("Product", StringType(), False),
    StructField("Region", StringType(), False),
    StructField("Quantity", IntegerType(), False),
    StructField("Amount", DoubleType(), False),
    StructField("OrderDate", TimestampType(), False)
])

# Create DataFrame
df_sales = spark.createDataFrame(sample_data, schema)
display(df_sales.limit(10))
print(f"Total records: {df_sales.count()}")

## 4. Writing Data to OneLake

Demonstrate different ways to write data to OneLake:

In [ ]:
# Write as Delta table (recommended for OneLake)
# This provides ACID transactions, time travel, and optimal performance
delta_table_name = "sales_demo"

# Write to Delta table
df_sales.write.format("delta") \
    .mode("overwrite") \
    .save(f"Tables/{delta_table_name}")

print(f"Data written to Delta table: {delta_table_name}")

In [ ]:
# Write as Parquet files
df_sales.write.mode("overwrite") \
    .parquet("Files/sales_parquet")

print("Data written to Parquet format")

## 5. Querying Data with SQL

OneLake supports SQL queries through Spark SQL:

In [ ]:
# Register DataFrame as temp view
df_sales.createOrReplaceTempView("sales")

# Run SQL query
result = spark.sql("""
    SELECT 
        Region,
        Product,
        COUNT(*) as OrderCount,
        SUM(Quantity) as TotalQuantity,
        ROUND(SUM(Amount), 2) as TotalRevenue
    FROM sales
    GROUP BY Region, Product
    ORDER BY TotalRevenue DESC
""")

display(result)

## 6. Data Transformations

Demonstrate common data transformation operations:

In [ ]:
# Add calculated columns
df_enriched = df_sales \
    .withColumn("UnitPrice", round(col("Amount") / col("Quantity"), 2)) \
    .withColumn("Year", year(col("OrderDate"))) \
    .withColumn("Month", month(col("OrderDate"))) \
    .withColumn("Quarter", quarter(col("OrderDate")))

display(df_enriched.limit(10))

In [ ]:
# Aggregations
monthly_summary = df_enriched \
    .groupBy("Year", "Month", "Region") \
    .agg(
        count("*").alias("OrderCount"),
        sum("Quantity").alias("TotalQuantity"),
        round(sum("Amount"), 2).alias("TotalRevenue"),
        round(avg("Amount"), 2).alias("AvgOrderValue")
    ) \
    .orderBy("Year", "Month", "Region")

display(monthly_summary)

## 7. Working with OneLake Shortcuts

OneLake shortcuts allow you to reference data from external sources without copying it:

### Types of Shortcuts:
- **OneLake shortcuts**: Reference data in other OneLake locations
- **ADLS Gen2 shortcuts**: Reference Azure Data Lake Storage
- **S3 shortcuts**: Reference AWS S3 buckets

Shortcuts are created through the Fabric UI:
1. Navigate to your Lakehouse
2. Right-click on Files or Tables
3. Select "New shortcut"
4. Choose the source type and configure connection

In [ ]:
# Once shortcuts are created, they appear as regular folders/tables
# Example: reading from a shortcut
# df_shortcut = spark.read.format("delta").load("Tables/external_data_shortcut")
# display(df_shortcut.limit(10))

print("Shortcuts provide seamless access to external data without data movement")

## 8. Delta Lake Features

OneLake uses Delta Lake format which provides advanced capabilities:

In [ ]:
# Time Travel - Query historical versions of data
# df_version = spark.read.format("delta") \
#     .option("versionAsOf", 0) \
#     .load(f"Tables/{delta_table_name}")
# display(df_version)

print("Delta Lake supports time travel to query historical data versions")

In [ ]:
# Get table history
from delta.tables import DeltaTable

# dt = DeltaTable.forPath(spark, f"Tables/{delta_table_name}")
# history_df = dt.history()
# display(history_df)

print("Delta tables maintain complete history of all operations")

## 9. Best Practices for OneLake

### Storage Best Practices:
1. **Use Delta tables** for structured data
2. **Partition large tables** by frequently filtered columns (e.g., date)
3. **Use appropriate file sizes** (128MB-1GB per file)
4. **Leverage shortcuts** to avoid data duplication

### Performance Best Practices:
1. **Use Z-ORDER** for multi-dimensional clustering
2. **Optimize file layout** with OPTIMIZE command
3. **Cache frequently accessed data**
4. **Use broadcast joins** for small dimension tables

In [ ]:
# Example: Optimize Delta table
# spark.sql(f"OPTIMIZE delta.`Tables/{delta_table_name}`")
# spark.sql(f"OPTIMIZE delta.`Tables/{delta_table_name}` ZORDER BY (Region, Product)")

print("OPTIMIZE command compacts small files and improves query performance")

## 10. Data Governance in OneLake

OneLake integrates with Microsoft Purview for comprehensive data governance:

- **Data lineage**: Track data flow across your organization
- **Data classification**: Automatically classify sensitive data
- **Access control**: Manage permissions at workspace and item level
- **Data discovery**: Find and understand available datasets

These features are configured through the Fabric portal and Purview governance center.

## Summary

In this notebook, we covered:
- ✅ OneLake architecture and benefits
- ✅ Reading and writing data in multiple formats
- ✅ Creating and querying Delta tables
- ✅ Data transformations and aggregations
- ✅ OneLake shortcuts for external data
- ✅ Delta Lake advanced features
- ✅ Best practices for storage and performance
- ✅ Data governance capabilities

## Next Steps
- Explore Real-Time Intelligence for streaming data
- Learn about Power BI semantic models
- Implement data pipelines for orchestration
- Set up CI/CD for your Fabric solutions